In [1]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_community.llms.ollama import Ollama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

C:\Users\chira\AppData\Local\Temp\ipykernel_5752\1707036427.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.llms.ollama import Ollama


In [53]:
import json
file_path = "../data/cleaned_ahm_housing_data.json"
# Load property data
def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return json.load(f)

data = load_data(file_path)
len(data)

5819

In [54]:
from langchain.docstore.document import Document
documents = []
for x in data:
    content = str(x)
    documents.append(Document(page_content=content, metadata={"url": x["url"]}))
len(documents)

5819

In [55]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")

In [56]:
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("property_vector_db")

In [57]:
# Initialize vector_store & LLM
vector_store = FAISS.load_local("property_vector_db",  embeddings)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


In [2]:
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [7]:
llm.invoke("Tell me joke")

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45, 'completion_time': 0.026666667, 'prompt_time': 0.00538613, 'queue_time': 0.178578184, 'total_time': 0.032052797}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-e4cdfea1-d33e-4995-9a35-56f38b3bbd3f-0', usage_metadata={'input_tokens': 13, 'output_tokens': 32, 'total_tokens': 45})

In [78]:
initial_query = f"""Hello, I am looking for a property in Ahmedabad.  
My budget is between ₹50L - ₹1Cr.  
I am interested in properties in the following areas: Shela, Shilaj, and South Bopal.  
The property configuration should be either 2 BHK or 3 BHK.  
Please provide only the budget, address, and property configuration details in the response."""

In [79]:
from typing import List, Dict, Union
from pydantic import BaseModel, Field

class Budget(BaseModel):
    min: int = Field(description="Minimum budget")
    max: int = Field(description="Maximum budget")

class UserPreference(BaseModel):
    budget: Budget = Field(description="Budget range for property purchase")
    address: List[str] = Field(description="Preferred locations for property purchase")
    property_config: List[str] = Field(description="Property configuration preferences (e.g., 2 BHK, 3 BHK)")


# class UserPreference(BaseModel):
#     budget: Dict[str, int] = Field(description="Budget range for property purchase")
#     address: List[str] = Field(description="Preferred locations for property purchase")
#     property_config: Dict[str, List[str]] = Field(description="Property configuration preferences (e.g., 2 BHK, 3 BHK)")
#     property_type: str = Field(description="Type of property (e.g., apartment, villa)", default="apartment")
#     possession: Dict[str, int] = Field(description="Possession time in months")

structured_llm = llm.with_structured_output(UserPreference)
response = structured_llm.invoke(initial_query)
response

UserPreference(budget=Budget(min=50, max=100), address=['Shela', 'Shilaj', 'South Bopal'], property_config=['2 BHK', '3 BHK'])

In [25]:
print(response.budget)
print(response.address)
print(response.property_config)

{'gte': 5000000, 'lte': 10000000}
['Shela', 'Shilaj', 'South Bopal']
[{'bhk': 2}, {'bhk': 3}]


In [10]:
user_query = {
    "query": "I am looking for a 3 BHK apartment in Ahmedabad with a budget of 90L.",
    "budget": "60L - 90L",
    "location": "Ahmedabad",
    "property_type": "Apartment",
    "bhk": "2",
    "amenities": "Gym, Security, Parking",
    "proximity": "Near Schools, Hospitals",
    "loan_emi": "Willing to take a loan",
    "possession": "Ready to Move"
}
retriever.get_relevant_documents(str(user_query))

C:\Users\chira\AppData\Local\Temp\ipykernel_22864\967772629.py:12: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(str(user_query))


[Document(id='a43ed9c9-0a79-4657-bfc2-c9efd77f5921', metadata={'url': 'https://housing.com/buy-pramukh-shypram-sky-by-pramukh-nirman-assets-llp-in-chanakyapuri-ahmedabad-pid-261963'}, page_content="{'url': 'https://housing.com/buy-pramukh-shypram-sky-by-pramukh-nirman-assets-llp-in-chanakyapuri-ahmedabad-pid-261963', 'name': 'Pramukh Shypram Sky', 'developer': 'Pramukh Nirman Assets LLP', 'address': 'Uma Bunglows, Opp. Ganesh Park, ,Shukan Bunglow Road, Chanakyapuri, North West, Ahmedabad', 'price_range': '1.2 Cr - 1.62 Cr', 'basic_details': {'Configurations': '3, 4 BHK Apartments', 'Possession Starts': 'Mar, 2025', 'Avg. Price': '₹1.15 Lacs - 1.3 Lacs/sq.mt'}, 'nearby_places': [{'Category': 'School', 'Name': 'I D Patel School', 'Travel Time': '1 min', 'Distance': '(0.6 km)'}, {'Category': 'Hospital', 'Name': 'Shayona Hospital & ICU', 'Travel Time': '2 mins', 'Distance': '(1.4 km)'}, {'Category': 'Restaurant', 'Name': 'Friesbag', 'Travel Time': '2 mins', 'Distance': '(1 km)'}, {'Catego

In [7]:
system_prompt = (
    "You are an realestate advisor for advising user to buy properties based on their preference. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Show all retrived context to user and explain everything about properties and give analysis why should they buy it"
    "\n\n"
    "{context}"
)

In [83]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(structured_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [13]:
initial_query = f"""Hello, I am finding property in Ahmedabad.  
The possession should be within 3 years. 
I am looking for Only New Construction. 
Property type is apartment and configuration can be [2 BKH, 3 BHK]. 
I am finding properties in the following areas: [Shela, Shilaj, south bopal].  
My budget is ₹50L - ₹1Cr. 
I want to have Quiet & Residential neighborhood."""

In [12]:
user_query = {
    "price": "between 60L  to 90L",
    "location": "Ahmedabad",
    "property_type": "Apartment",
    "BHK Type": ["2 BHK", "3 BHK"],
    "amenities": "Gym, Security, Parking",
    "proximity": "Near Schools, Hospitals",
    "possession": "within 3 years",
    "address": ["Shela", "South Bopal", "Shilaj"]
}

In [84]:
response = rag_chain.invoke({"input": str(initial_query)})
response

BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [62]:
from IPython.display import display, Markdown

In [85]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = ("""
    Given a chat history and the latest user question  which might reference context in the chat history,    
    formulate a standalone question which can be understood 
    without the chat history and if necessary use above format for retrival to formulate a standalone question. Do NOT answer the question,
    "just reformulate it if needed and otherwise return it as is."""
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [86]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [34]:
Initial_suggestion = {
    "City": "Ahmedabad",
    "Possession": "In 3 years",
    "Property": "Only New Construction",
    "Property type": "Apartment",
    "Property Configuration": ["2 BHK", "3 BHK"],
    "Address": ["Shela", "Shilaj", "Bopal", "South Bopal"],
    "Workplace/School": "Yes, should be within nearby are",
    "Budget": "₹50L - ₹1Cr",
    "neighbourhood": "Quiet & Residential",
}

In [87]:
chat_history = []

first_question = "Hi, can you suggest me properties in shela aroung 50L to 60L?"
ai_msg_1 = rag_chain.invoke({"input": str("first_question"), "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=str(first_question)),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

In [88]:
ai_msg_1

{'input': 'first_question',
 'chat_history': [HumanMessage(content='Hi, can you suggest me properties in shela aroung 50L to 60L?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm happy to help you with your question! However, I need some more information from you. Could you please specify which property you would like me to analyze and provide a recommendation for? The options are:\n\n1. Western Unity by Western Buildcon\n2. Anmol Forum by Omro Developers\n3. Prerna Darshan by Prerna Infrabuild Ltd.\n4. Iscon JP Iscon Residency by Iscon Group Builders\n5. Shyam Jivan Amrut by Shyam Jivan Developers\n\nPlease let me know which one you are interested in, and I'll do my best to provide you with an analysis and a recommendation.", additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'url': 'https://housing.com/buy-western-unity-by-western-buildcon-in-vaishno-devi-circle-ahmedabad-pid-292027'}, page_content='{\'url\': \'https://housing.com/buy

In [67]:
display(Markdown(ai_msg_1["answer"]))

Based on the retrieved context, I will analyze the properties and provide an answer to the question.

You are considering buying a property in Ahmedabad. I will analyze the properties mentioned in the context and provide my recommendation.

The first property I will analyze is "Western Unity" by Western Buildcon in Vaishno Devi Circle, Ahmedabad.

**Property Details:**

* Address: Nr Sardar Dham Vaishnudevi Circle, Khodiyar, North West, Ahmedabad
* Price Range: ₹63.76 L - ₹72.0 L
* Configuration: 3 BHK Apartment
* Possession Starts: Dec, 2026
* Nearby Places:
	+ School: Gems Genesis International School (GGIS) - 3 mins
	+ Hospital: KD Hospital - 2 mins
	+ Restaurant: "VEDA" GATHIYA RATH - Unknown
	+ Mall: Adani Shantigram Mall - 8 mins
	+ Bus Stand: Godrej Garden City Bus Stop - 6 mins

**Analysis:**

* Pros:
	+ Located in a well-connected area with easy access to schools, hospitals, and malls.
	+ Possession starts in Dec, 2026, which is relatively soon.
* Cons:
	+ Traffic congestion in the area.
	+ High rental charges.
	+ Lack of parking facilities.

**Recommendation:**

I recommend considering "Western Unity" as an option for buying a property in Ahmedabad. The property's location, with easy access to schools, hospitals, and malls, makes it a convenient choice. However, the traffic congestion and high rental charges are drawbacks to consider. It's essential to weigh these pros and cons before making a decision.

Please let me know if you would like me to analyze the other properties as well.

In [44]:
second_question = "can you give me details about Pramukh Prathna Residency?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

In [46]:
chat_history.extend(
    [
        HumanMessage(content=str(second_question)),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)

In [45]:
display(Markdown(ai_msg_2["answer"]))

Pramukh Prathna Residency is a residential project located in Chanakyapuri, Ahmedabad. Here are some details about the project:

**Project Details**

* Project Name: Pramukh Prathna Residency
* Location: Chanakyapuri, Ahmedabad
* Developer: Pramukh Nirman Assets LLP
* Launched Date: July 2021
* Possession Date: March 2025
* Project Size: 1.68 Acres
* Number of Buildings: 3
* Number of Units: 161
* Configuration: 3, 4 BHK Apartments
* Sizes: 104.61 - 124.58 sq.mt.

**Amenities**

* Power Backup
* Fire Sprinklers
* Landscaping & Tree Planting
* Gyms
* Swimming Pool
* Children's Play Area
* Clubhouse
* Library
* Indoor Games
* Outdoor Games
* Multi-Purpose Court
* Security with CCTV cameras
* 24/7 Power Backup

**Nearby Amenities**

* Schools: I D Patel School (1 min), S S Divine School (3 min)
* Hospitals: Shayona Hospital & ICU (2 min), SPS Apollo Hospital (5 min)
* Restaurants: Friesbag (2 min), Pause Restrocafe (2 min)
* Malls: Devnandan Complex (2 min), Shukan Bunglow Road (5 min)
* Bus Stand: Ranna Park Bus Stop (4 min)

**Connectivity**

* Road Connectivity: Chanakyapuri is connected to other parts of the city via the Sardar Patel Ring Road and the Ahmedabad-Vadodara Expressway.
* Public Transport: The project is close to public transport hubs like Ranna Park Bus Stop and the Ahmedabad Metro.

**Price Range**

* The price range for Pramukh Prathna Residency is ₹1.20 Cr - ₹1.62 Cr, depending on the configuration and size of the apartment.

**Ratings**

* Connectivity: 5/5
* Neighborhood: 4.8/5
* Safety: 5/5
* Livability: 4.5/5

**Good Points**

* Safe for children
* Good school nearby
* No water shortage
* Good connectivity to other parts of the city

**Bad Points**

* Low security
* High price for area

Please note that these details are based on the retrieved context and may not be exhaustive or up-to-date. It's always a good idea to verify the information with the developer or a reliable source before making a decision.

In [47]:
question = "But it is not in shela. I told you to suggest me properties in shela?"
ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=str(question)),
        AIMessage(content=ai_msg["answer"]),
    ]
)
display(Markdown(ai_msg["answer"]))

I apologize for the mistake. I misunderstood the location you were looking for. I do not have any properties in Shela matching the price range of 50-60 lakhs. However, I can suggest you to check the properties in Shela on real estate websites like Housing.com, 99acres.com, or Magicbricks.com. You can also contact local real estate agents or brokers who have knowledge of the area and can guide you through the process.

Please let me know if there's anything else I can help you with. I'll make sure to provide more accurate information in the future.

In [48]:
question = "Okay than suggest me properties in shela?"
ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=str(question)),
        AIMessage(content=ai_msg["answer"]),
    ]
)
display(Markdown(ai_msg["answer"]))

I apologize for the earlier mistake. After re-checking the retrieved context, I found the following property in Shela:

* **Hindva Shantiniketan Solitare**: This is a project by Hindva Builders in Nikol, Ahmedabad, which is close to Shela. The property offers 2 BHK apartments with a size range of 1161-1500 sq ft. The price range is not specified in the context, but it's within the range of 50-60 lakhs.

Here are some details about the property:

* Location: Nikol, Ahmedabad (close to Shela)
* Developer: Hindva Builders
* Property Type: 2 BHK apartments
* Size Range: 1161-1500 sq ft
* Amenities: Car parking spaces, children's play area, club house, gymnasium, landscaped gardens, lifts, power backup, regular supply of water
* Ratings: Connectivity - 4.6/5, Neighborhood - 4.5/5, Safety - 4.6/5, Livability - 4.7/5
* Good Points: Accessible amenities nearby, peaceful locality, high security standard
* Bad Points: Ban loudspeakers by hawkers, high pollution levels, high rents in the city

Please note that this is the only property in Shela that matches your price range. I recommend verifying the information with the developer or a reliable source before making a decision.

If you'd like to explore more options, I can suggest you to check the properties in Shela on real estate websites like Housing.com, 99acres.com, or Magicbricks.com. You can also contact local real estate agents or brokers who have knowledge of the area and can guide you through the process.

In [49]:
question = "There is property named Vishwanath Sarathya West.don't you know about it?"
ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=str(question)),
        AIMessage(content=ai_msg["answer"]),
    ]
)
display(Markdown(ai_msg["answer"]))

I apologize for not knowing about Vishwanath Sarathya West earlier. As I'm a large language model, my training data is based on the information I was provided, and not all properties or information may be included.

However, since you've mentioned Vishwanath Sarathya West, I'd like to know more about it. Can you please provide me with more information about this property, such as its location, price range, and amenities? This will help me better understand your requirements and provide a more accurate response.